In [9]:
"""
This notebook uses the folium library to plot an interactive map for the trucks' routes.

It uses the Google Directions API to get the route for each list of waypoints.
"""
import folium
from folium.features import DivIcon
import pandas as pd
import googlemaps 
import polyline
from datetime import datetime
from os import listdir, getcwd
from os.path import isfile, join

# This is used to read in the Google Maps API key.
with open("API_key.txt", "r") as f:
    GMAP_API_KEY = f.read()

# function reads a route file and adds the contents as tuples (containing coordinates) to a list, returns that list
def read_file(fileName):

    coords = []
    for line in pd.read_csv(fileName, encoding='utf-8', chunksize=1):
        coords.append((line.iloc[0,0], line.iloc[0,1]))
    
    return coords

# function creates and returns a numbered icon for the folium marker
def create_divcon(num):

    icon = DivIcon(icon_size=(150,36), 
                   icon_anchor=(14,40),
                   html='<div style="font-size: 18pt; color : black">' + '{:01d}'.format(num+1) + '</div>')  # styling for icon
    
    return icon

# function takes in a list of any size (at least two items) containing gps coordinates as tuples
# order: latitude, longitude
def generate_map(fileList):

    # Read in route files for each truck and store the coordinates.
    coord_list = []

    for f in fileList:
        coord_list.append(read_file(f))

        
    # initialize map from starting point (first tuple in list). Since all of our addresses are close by, this should be sufficient.
    m = folium.Map(location=[coord_list[0][0][0], coord_list[0][0][1]])

    # Set up the Google Maps API and
    curDate = datetime.now()
    gmaps = googlemaps.Client(key=GMAP_API_KEY)

    points = []

    # We'll colour code the polyline of each different truck so we can tell them apart. Folium has many colours, so this could be done for many more.
    colours = ["red", 'blue', 'green']

    for i, routes in enumerate(coord_list):

        # Create a map marker for every route and number it.
        for j in range(len(routes)):
            #points.append([routes[j][0], routes[j][1]])
            folium.Marker([routes[j][0], routes[j][1]],
                        icon=create_divcon(j)
                        ).add_to(m)
    
        # Call the Google Directions API to generate a polyline for the route. We'll give it a unique colour for each mail truck.
        polyline_coords = []

        for k in range(0, len(routes) - 1):
            result = gmaps.directions(origin = routes[k], destination = routes[k+1], mode='driving',units= 'metric', traffic_model='best_guess', departure_time = curDate)

            for x in result[0]['legs'][0]['steps']:
                polyline_coords.extend(polyline.decode(x['polyline']['points']))

        # add a line to connect the points
        folium.PolyLine(polyline_coords, color=colours[i]).add_to(m)

    # We return our map.
    return m


In [10]:
# Read in a list of all CSV files in the CWD, which we're assuming are exclusively route information.
path = getcwd()
fileList = [path + '/' + f for f in listdir(path) if (isfile(join(path, f)) and ".csv" in f)]

# Pass the files to our map generator, which will read them in and make the map, which we display.
m = generate_map(fileList)
m